In [ ]:
1from google.colab import drive

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn as skl
import sklearn.svm

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/project/utils.py /content/utils.py
!pip install python-dotenv
import utils

In [ ]:
tracks = utils.load('/content/drive/MyDrive/project/fma_metadata/tracks.csv')
genres = utils.load('/content/drive/MyDrive/project/fma_metadata/genres.csv')
features = utils.load('/content/drive/MyDrive/project/fma_metadata/features.csv')
echonest = utils.load('/content/drive/MyDrive/project/fma_metadata/echonest.csv')

In [ ]:
genres.shape

(163, 4)

In [ ]:
small = tracks['set', 'subset'] <= 'small'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

y_train = tracks.loc[small & train, ('track', 'genre_top')]
y_test = tracks.loc[small & test, ('track', 'genre_top')]
y_val = tracks.loc[small & val, ('track', 'genre_top')]
X_train = features.loc[small & train, 'mfcc']
X_val = features.loc[small & val, 'mfcc']
X_test = features.loc[small & test, 'mfcc']

print('{} training examples, {} testing examples'.format(y_train.size, y_test.size))
print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

6400 training examples, 800 testing examples
140 features, 8 classes


In [ ]:
deleted = [99134, 108925, 133297, 98565, 98567, 98569]
X_train.drop(labels = deleted, axis = 0, inplace = True)
y_train.drop(labels = deleted, axis = 0, inplace = True)

In [ ]:
X_TRAIN = echonest.loc[small & train, 'echonest']
X_TEST = echonest.loc[small & test, 'echonest']
X_VAL = echonest.loc[small & val, 'echonest']
X_TRAIN = X_TRAIN.loc[:, 'audio_features']
X_TEST = X_TEST.loc[:, 'audio_features']
X_VAL = X_VAL.loc[:, 'audio_features']
train_indexes = np.array(X_TRAIN.index)
test_indexes = np.array(X_TEST.index)
val_indexes = np.array(X_VAL.index)
echo_Y_train = y_train[train_indexes]
echo_Y_test = y_test[test_indexes]
echo_Y_val = y_val[val_indexes]

In [ ]:
# np.unique(echonest.loc[small & train].keys())
# np.unique(features.loc[small & train].keys())#['mfcc']

In [ ]:
def normalize(frame):
    min_val = frame.min()
    max_val = frame.max()
    frame = (frame - min_val) / (max_val - min_val)
    return frame

X_train_N = normalize(X_train)
X_test_N = normalize(X_test)
X_val_N = normalize(X_val)
X_TRAIN_N = normalize(X_TRAIN)
X_TEST_N = normalize(X_TEST)
X_VAL_N = normalize(X_VAL)

In [ ]:
list(y_train.cat.categories)

['Blues',
 'Classical',
 'Country',
 'Easy Listening',
 'Electronic',
 'Experimental',
 'Folk',
 'Hip-Hop',
 'Instrumental',
 'International',
 'Jazz',
 'Old-Time / Historic',
 'Pop',
 'Rock',
 'Soul-RnB',
 'Spoken']

In [ ]:
dct = {
 'Blues':                0,
 'Classical':            1,
 'Country':              2,
 'Easy Listening':       3,
 'Electronic':           4,
 'Experimental':         5,
 'Folk':                 6,
 'Hip-Hop':              7,
 'Instrumental':         8,
 'International':        9,
 'Jazz':                 10,
 'Old-Time / Historic':  11,
 'Pop':                  12,
 'Rock':                 13,
 'Soul-RnB':             14,
 'Spoken':               15
}
train_y = dict()
test_y = dict()
val_y = dict()
for i in y_train.keys():
    train_y[i] = dct[y_train[i]]
for i in y_test.keys():
    test_y[i] = dct[y_test[i]]
for i in y_val.keys():
    val_y[i] = dct[y_val[i]]

In [ ]:
lin_reg_y_train = np.array(list(train_y.values()))
lin_reg_y_test = np.array(list(test_y.values()))
lin_reg_y_val = np.array(list(val_y.values()))

In [ ]:
np.unique(y_train)

array(['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental',
       'International', 'Pop', 'Rock'], dtype=object)

#TELEMETRIES

In [ ]:
# Be sure training samples are shuffled.
X_train, y_train = skl.utils.shuffle(X_train, y_train, random_state=42)

# Standardize features by removing the mean and scaling to unit variance.
scaler = skl.preprocessing.StandardScaler(copy=False)
scaler.fit_transform(X_train)
scaler.transform(X_test)

def train_poly(tol, deg):
    clf = skl.svm.SVC(kernel = 'poly', tol = tol, degree = deg)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    return score

def train_rbf(tol):
    clf = skl.svm.SVC(kernel = 'rbf', tol = tol)
    clf.fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    return score

print('SVM Polynomial Kernel: ')
for i in (1, 2, 3, 4):
    for j in (1e0, 1e-1, 1e-2, 1e-3):
        print('degree = %d, tolerance = %f, accuracy = %.2f'%(i, j, 100*train_poly(i, j)))

print('\nSVM rbf Kernel: ')
for i in (1e0, 1e-1, 1e-2, 1e-3):
    print('tolerance = %f, accuracy = %.2f'%(i, 100*train_rbf(i)))

SVM Polynomial Kernel: 
degree = 1, tolerance = 1.000000, accuracy = 42.62
degree = 1, tolerance = 0.100000, accuracy = 12.50
degree = 1, tolerance = 0.010000, accuracy = 12.50
degree = 1, tolerance = 0.001000, accuracy = 12.50
degree = 2, tolerance = 1.000000, accuracy = 41.88
degree = 2, tolerance = 0.100000, accuracy = 12.50
degree = 2, tolerance = 0.010000, accuracy = 12.50
degree = 2, tolerance = 0.001000, accuracy = 12.50
degree = 3, tolerance = 1.000000, accuracy = 12.50
degree = 3, tolerance = 0.100000, accuracy = 12.50
degree = 3, tolerance = 0.010000, accuracy = 12.50
degree = 3, tolerance = 0.001000, accuracy = 12.50
degree = 4, tolerance = 1.000000, accuracy = 12.50
degree = 4, tolerance = 0.100000, accuracy = 12.50
degree = 4, tolerance = 0.010000, accuracy = 12.50
degree = 4, tolerance = 0.001000, accuracy = 12.50

SVM rbf Kernel: 
tolerance = 1.000000, accuracy = 46.50
tolerance = 0.100000, accuracy = 46.62
tolerance = 0.010000, accuracy = 46.62
tolerance = 0.001000, acc

In [ ]:
clf = skl.svm.SVC(kernel = 'rbf', tol = 0.01)
clf.fit(X_train, y_train)
clf.score(X_val, y_val)

0.125

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

for i in range(1,50):
    neigh = KNeighborsClassifier(n_neighbors=i*2+1)
    neigh.fit(X_train, y_train)
    score = neigh.score(X_test,y_test)
    print('kNN accuracy when k = ',i*2+1,': {:.2%}'.format(score))

neigh.score(X_val, y_val)

kNN accuracy when k =  3 : 33.75%
kNN accuracy when k =  5 : 34.88%
kNN accuracy when k =  7 : 35.25%
kNN accuracy when k =  9 : 35.75%
kNN accuracy when k =  11 : 36.50%
kNN accuracy when k =  13 : 36.00%
kNN accuracy when k =  15 : 36.25%
kNN accuracy when k =  17 : 36.62%
kNN accuracy when k =  19 : 37.00%
kNN accuracy when k =  21 : 36.12%
kNN accuracy when k =  23 : 36.25%
kNN accuracy when k =  25 : 36.50%
kNN accuracy when k =  27 : 37.12%
kNN accuracy when k =  29 : 37.88%
kNN accuracy when k =  31 : 37.88%
kNN accuracy when k =  33 : 37.38%
kNN accuracy when k =  35 : 37.75%
kNN accuracy when k =  37 : 37.75%
kNN accuracy when k =  39 : 38.75%
kNN accuracy when k =  41 : 37.62%
kNN accuracy when k =  43 : 36.75%
kNN accuracy when k =  45 : 37.25%
kNN accuracy when k =  47 : 37.12%
kNN accuracy when k =  49 : 36.25%
kNN accuracy when k =  51 : 35.50%
kNN accuracy when k =  53 : 36.12%
kNN accuracy when k =  55 : 36.88%
kNN accuracy when k =  57 : 36.62%
kNN accuracy when k =  5

0.12875

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
score = clf.score(X_test, y_test)
# score = clf.score(X_val, y_val)
print('Gaussian naive bayes accuracy: {:.2%}'.format(score))

Gaussian naive bayes accuracy: 36.00%


In [ ]:
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(X_train_N, lin_reg_y_train)
score = clf.score(X_test_N, lin_reg_y_test)
# score = clf.score(X_val_N, lin_reg_y_val)
print('Categorical naive bayes accuracy: {:.2%}'.format(score))

Categorical naive bayes accuracy: 32.12%


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train_N, lin_reg_y_train)
score = clf.score(X_test_N, lin_reg_y_test)
# score = clf.score(X_val_N, lin_reg_y_val)
print('Multinomial naive bayes accuracy: {:.2%}'.format(score))

Multinomial naive bayes accuracy: 34.50%


In [ ]:
from sklearn.linear_model import LogisticRegression
# reg = LogisticRegression(multi_class='multinomial', max_iter = 100).fit(X_train_N, lin_reg_y_train)
# score = reg.score(X_test_N, lin_reg_y_test)
# print('Logistic regression accuracy: {:.2%}'.format(score))
# reg = LogisticRegression(multi_class='multinomial', max_iter = 175).fit(X_train_N, lin_reg_y_train)
# score = reg.score(X_test_N, lin_reg_y_test)
# print('Logistic regression accuracy: {:.2%}'.format(score))
# reg = LogisticRegression(multi_class='multinomial', max_iter = 200).fit(X_train_N, lin_reg_y_train)
# score = reg.score(X_test_N, lin_reg_y_test)
# print('Logistic regression accuracy: {:.2%}'.format(score))
# reg = LogisticRegression(multi_class='multinomial', max_iter = 225).fit(X_train_N, lin_reg_y_train)
# score = reg.score(X_test_N, lin_reg_y_test)
# print('Logistic regression accuracy: {:.2%}'.format(score))
reg = LogisticRegression(multi_class='multinomial', max_iter = 500).fit(X_train_N, lin_reg_y_train)
score = reg.score(X_test_N, lin_reg_y_test)
print('Logistic regression accuracy: {:.2%}'.format(score))
score = reg.score(X_val_N, lin_reg_y_val)
print('Logistic regression accuracy: {:.2%}'.format(score))

Logistic regression accuracy: 40.62%
Logistic regression accuracy: 44.25%


In [ ]:
from sklearn.linear_model import Perceptron
reg = Perceptron().fit(X_train_N, lin_reg_y_train)
score = reg.score(X_test_N, lin_reg_y_test)
score = reg.score(X_val_N, lin_reg_y_val)
print('Perceptron accuracy: {:.2%}'.format(score))

Perceptron accuracy: 35.25%


#OYNAKLIK


In [ ]:
# Be sure training samples are shuffled.
X_TRAIN_N, echo_Y_train = skl.utils.shuffle(X_TRAIN_N, echo_Y_train, random_state=42)

# Standardize features by removing the mean and scaling to unit variance.
scaler = skl.preprocessing.StandardScaler(copy=False)
scaler.fit_transform(X_TRAIN_N)
scaler.transform(X_TEST_N)

def train_poly(tol, deg):
    clf = skl.svm.SVC(kernel = 'poly', tol = tol, degree = deg)
    clf.fit(X_TRAIN_N, echo_Y_train)
    score = clf.score(X_TEST_N, echo_Y_test)
    return score

def train_rbf(tol):
    clf = skl.svm.SVC(kernel = 'rbf', tol = tol)
    clf.fit(X_TRAIN_N, echo_Y_train)
    score = clf.score(X_TEST_N, echo_Y_test)
    return score

print('SVM Polynomial Kernel: ')
for i in (1, 2, 3, 4):
    for j in (1e0, 1e-1, 1e-2, 1e-3):
        print('degree = %d, tolerance = %f, accuracy = %.2f'%(i, j, 100*train_poly(i, j)))

print('\nSVM rbf Kernel: ')
for i in (1e0, 1e-1, 1e-2, 1e-3):
    print('tolerance = %f, accuracy = %.2f'%(i, 100*train_rbf(i)))

SVM Polynomial Kernel: 
degree = 1, tolerance = 1.000000, accuracy = 53.44
degree = 1, tolerance = 0.100000, accuracy = 5.29
degree = 1, tolerance = 0.010000, accuracy = 5.29
degree = 1, tolerance = 0.001000, accuracy = 5.29
degree = 2, tolerance = 1.000000, accuracy = 31.75
degree = 2, tolerance = 0.100000, accuracy = 5.29
degree = 2, tolerance = 0.010000, accuracy = 5.29
degree = 2, tolerance = 0.001000, accuracy = 5.29
degree = 3, tolerance = 1.000000, accuracy = 18.52
degree = 3, tolerance = 0.100000, accuracy = 18.52
degree = 3, tolerance = 0.010000, accuracy = 18.52
degree = 3, tolerance = 0.001000, accuracy = 18.52
degree = 4, tolerance = 1.000000, accuracy = 18.52
degree = 4, tolerance = 0.100000, accuracy = 18.52
degree = 4, tolerance = 0.010000, accuracy = 18.52
degree = 4, tolerance = 0.001000, accuracy = 18.52

SVM rbf Kernel: 
tolerance = 1.000000, accuracy = 51.85
tolerance = 0.100000, accuracy = 59.26
tolerance = 0.010000, accuracy = 59.26
tolerance = 0.001000, accuracy 

In [ ]:
clf = skl.svm.SVC(kernel = 'rbf', tol = 0.1)
clf.fit(X_TRAIN_N, echo_Y_train)
clf.score(X_VAL_N, echo_Y_val)

0.5932203389830508

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

for i in range(1,50):
    neigh = KNeighborsClassifier(n_neighbors=i*2+1)
    neigh.fit(X_TRAIN_N, echo_Y_train)
    score = neigh.score(X_TEST_N, echo_Y_test)
    print('kNN accuracy when k = ',i*2+1,': {:.2%}'.format(score))

kNN accuracy when k =  3 : 48.68%
kNN accuracy when k =  5 : 47.62%
kNN accuracy when k =  7 : 51.32%
kNN accuracy when k =  9 : 54.50%
kNN accuracy when k =  11 : 52.38%
kNN accuracy when k =  13 : 53.97%
kNN accuracy when k =  15 : 55.03%
kNN accuracy when k =  17 : 56.08%
kNN accuracy when k =  19 : 53.44%
kNN accuracy when k =  21 : 58.20%
kNN accuracy when k =  23 : 56.08%
kNN accuracy when k =  25 : 55.56%
kNN accuracy when k =  27 : 56.08%
kNN accuracy when k =  29 : 54.50%
kNN accuracy when k =  31 : 55.56%
kNN accuracy when k =  33 : 55.56%
kNN accuracy when k =  35 : 57.67%
kNN accuracy when k =  37 : 57.14%
kNN accuracy when k =  39 : 57.14%
kNN accuracy when k =  41 : 57.14%
kNN accuracy when k =  43 : 54.50%
kNN accuracy when k =  45 : 56.61%
kNN accuracy when k =  47 : 55.56%
kNN accuracy when k =  49 : 54.50%
kNN accuracy when k =  51 : 55.56%
kNN accuracy when k =  53 : 55.03%
kNN accuracy when k =  55 : 53.97%
kNN accuracy when k =  57 : 53.97%
kNN accuracy when k =  5

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=21)
neigh.fit(X_TRAIN_N, echo_Y_train)
neigh.score(X_VAL_N, echo_Y_val)

0.5169491525423728

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_TRAIN_N, echo_Y_train)
score = clf.score(X_TEST_N, echo_Y_test)
# score = clf.score(X_VAL_N, echo_Y_val)
print('Gaussian naive bayes accuracy: {:.2%}'.format(score))

Gaussian naive bayes accuracy: 55.56%


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_TRAIN_N, echo_Y_train)
score = clf.score(X_TEST_N, echo_Y_test)
score = clf.score(X_VAL_N, echo_Y_val)
print('Multinomial naive bayes accuracy: {:.2%}'.format(score))

Multinomial naive bayes accuracy: 47.46%


In [ ]:
TRAIN_Y = dict()
TEST_Y = dict()
VAL_Y = dict()
for i in echo_Y_train.keys():
    TRAIN_Y[i] = dct[echo_Y_train[i]]
for i in echo_Y_test.keys():
    TEST_Y[i] = dct[echo_Y_test[i]]
for i in echo_Y_val.keys():
    VAL_Y[i] = dct[echo_Y_val[i]]
lin_reg_y_TRAIN = np.array(list(TRAIN_Y.values()))
lin_reg_y_TEST = np.array(list(TEST_Y.values()))
lin_reg_y_VAL = np.array(list(VAL_Y.values()))

In [ ]:
from sklearn.naive_bayes import ComplementNB
clf = ComplementNB()
clf.fit(X_TRAIN_N, lin_reg_y_TRAIN)
score = clf.score(X_TEST_N, lin_reg_y_TEST)
# score = clf.score(X_VAL_N, lin_reg_y_VAL)
print('Categorical naive bayes accuracy: {:.2%}'.format(score))

Categorical naive bayes accuracy: 50.79%


In [ ]:
from sklearn.linear_model import LogisticRegression
reg = LogisticRegression(multi_class='multinomial', max_iter = 10).fit(X_TRAIN_N, lin_reg_y_TRAIN)
score = reg.score(X_TEST_N, lin_reg_y_TEST)
print('10: Logistic regression accuracy: {:.2%}'.format(score))
reg = LogisticRegression(multi_class='multinomial', max_iter = 20).fit(X_TRAIN_N, lin_reg_y_TRAIN)
score = reg.score(X_TEST_N, lin_reg_y_TEST)
print('20: Logistic regression accuracy: {:.2%}'.format(score))
reg = LogisticRegression(multi_class='multinomial', max_iter = 50).fit(X_TRAIN_N, lin_reg_y_TRAIN)
score = reg.score(X_TEST_N, lin_reg_y_TEST)
print('50: Logistic regression accuracy: {:.2%}'.format(score))
score = reg.score(X_VAL_N, lin_reg_y_VAL)
print('Validation Logistic regression accuracy: {:.2%}'.format(score))
reg = LogisticRegression(multi_class='multinomial', max_iter = 75).fit(X_TRAIN_N, lin_reg_y_TRAIN)
score = reg.score(X_TEST_N, lin_reg_y_TEST)
print('75: Logistic regression accuracy: {:.2%}'.format(score))
reg = LogisticRegression(multi_class='multinomial', max_iter = 100).fit(X_TRAIN_N, lin_reg_y_TRAIN)
score = reg.score(X_TEST_N, lin_reg_y_TEST)
print('100: Logistic regression accuracy: {:.2%}'.format(score))

10: Logistic regression accuracy: 52.91%
20: Logistic regression accuracy: 54.50%
50: Logistic regression accuracy: 55.03%
Validation Logistic regression accuracy: 61.86%
75: Logistic regression accuracy: 54.50%
100: Logistic regression accuracy: 54.50%


In [ ]:
train_indexes.shape

In [ ]:
X_TRAIN

In [ ]:
X_TRAIN

,acousticness,danceability,energy,instrumentalness,liveness,speechiness,tempo,valence
track_id,,,,,,,,
2,0.416675,0.675894,0.634476,1.062807e-02,0.177647,0.159310,165.922,0.576661
5,0.043567,0.745566,0.701470,6.967990e-04,0.373143,0.124595,100.260,0.621661
10,0.951670,0.658179,0.924525,9.654270e-01,0.115474,0.032985,111.562,0.963590
140,0.376312,0.734079,0.265685,6.695811e-01,0.085995,0.039068,107.952,0.609991
141,0.963657,0.435933,0.075632,3.454935e-01,0.105686,0.026658,33.477,0.163950
...,...,...,...,...,...,...,...,...
123001,0.045738,0.472323,0.834652,2.280020e-05,0.249739,0.073384,77.513,0.179206
123002,0.241744,0.596352,0.579433,9.328010e-01,0.562251,0.027432,128.008,0.153574
123427,0.024610,0.337135,0.797854,7.110000e-08,0.158854,0.038150,167.926,0.657792


In [ ]:
X_TEST_N
echo_Y_test

In [ ]:
tracks

album  ...                                      track
         comments  ...                                      title
track_id           ...                                           
2               0  ...                                       Food
3               0  ...                               Electric Ave
5               0  ...                                 This World
10              0  ...                                    Freeway
20              0  ...                            Spiritual Level
...           ...  ...                                        ...
155316          0  ...                                  The Auger
155317          0  ...                            Let's Skin Ruby
155318          0  ...         My House Smells Like Kim Deal/Pulp
155319          0  ...                    The Man With Two Mouths
155320          0  ...  Another Trick Up My Sleeve (Instrumental)

[106574 rows x 52 columns]

In [ ]:
a = []
for i in features.keys():
    a.append(i[1])
np.unique(a)

array(['kurtosis', 'max', 'mean', 'median', 'min', 'skew', 'std'],
      dtype='<U8')

In [ ]:
train_indexes.shape

(987,)

In [ ]:
features.keys()

MultiIndex([('chroma_cens', 'kurtosis', '01'),
            ('chroma_cens', 'kurtosis', '02'),
            ('chroma_cens', 'kurtosis', '03'),
            ('chroma_cens', 'kurtosis', '04'),
            ('chroma_cens', 'kurtosis', '05'),
            ('chroma_cens', 'kurtosis', '06'),
            ('chroma_cens', 'kurtosis', '07'),
            ('chroma_cens', 'kurtosis', '08'),
            ('chroma_cens', 'kurtosis', '09'),
            ('chroma_cens', 'kurtosis', '10'),
            ...
            (    'tonnetz',      'std', '04'),
            (    'tonnetz',      'std', '05'),
            (    'tonnetz',      'std', '06'),
            (        'zcr', 'kurtosis', '01'),
            (        'zcr',      'max', '01'),
            (        'zcr',     'mean', '01'),
            (        'zcr',   'median', '01'),
            (        'zcr',      'min', '01'),
            (        'zcr',     'skew', '01'),
            (        'zcr',      'std', '01')],
           names=['feature', 'statistics', 